In [429]:
import pandas as pd
import pdfplumber
from collections import namedtuple

In [430]:
INV=namedtuple("INV",'vend_num vend_desc voucher invoice_number inv_date due_date inv_amount type_ net_amount validation Ref Description Code')

In [431]:
pdf_path="/Users/_senyonjo/Downloads/apreports.pdf"
with pdfplumber.open(pdf_path) as pdf:
    page=pdf.pages[15]
    text=page.extract_text()

In [432]:
print(text)

AccountsPayableReportPack
changeswillbereflectedunderthe"New"heading.Whenaninvoiceormanualcheckisdeleted,
theInvoiceAmountisautomaticallychangedtozeroandtheStatusfieldischangedto"D".
TotalTransactionCount Totalnumberoftransactionsonthelistincludingchangesanddeletions.Eachchangedtransaction
(i.e.,botholdandnew)countsasonetransaction.
1Notshownonsamplereport.
Invoice by Vendor List
Date: 08/25/2020 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032620 062620 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032720 062720 75.00 P 75.00 3 Postage * 1
3243387 072620 072620 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 082020 082520 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 082020 082520 239.00 P 239.00 2 Software Maintenanc

In [433]:
import re
vend_re=re.compile(r'\d{3} [A-Z].*')
for line in text.split("\n"):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
        print(vend_num,vend_desc)

200 UnitedParcelService
202 SoftwareTechnology,LLC
203 ClerkoftheCountyCourt
204 ClerkoftheDistrictCourt
225 Jackson/Wylinda
240 NELCO
250 D&BRealEstateManagementCompany
300 ThomsonReutersPaymentCenter
325 ProfessionalMessengerServices
400 SprintLocal&LongDistance
700 CleanAllJanitorialServices


In [434]:
line1items=[]
somedetails=re.compile(r'(^\d{3}\.\d{2}[A-Z].*) (\d{7}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)

    line=somedetails.search(line)
    # voucher invoice_number inv_date due_date type net_amount validation Ref Description Code
    if line:
        voucher=line.group(1)
        invoice_number=line.group(2)
        inv_date=line.group(3)
        due_date=line.group(4)
        inv_amount=line.group(5)
        type_=line.group(6)
        net_amount=line.group(7)
        validation=line.group(8)
        Ref=line.group(9)
        Description=line.group(10)
        Code=line.group(11)
        line1items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))

    




        





In [435]:
df=pd.DataFrame(line1items)
df

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,200,UnitedParcelService,200.01MLJ,3243387,032620,062620,75.00,P,75.00,,1,Overnight document delivery,1


In [436]:
line2items=[]
moredetails=re.compile(r'(^\d{7}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    line=moredetails.search(line)

    if line:
        print(line)
        invoice_number=line.group(1)
        inv_date=line.group(2)
        due_date=line.group(3)
        inv_amount=line.group(4)
        type_=line.group(5)
        net_amount=line.group(6)
        validation=line.group(7)
        Ref=line.group(8)
        Description=line.group(9)
        Code=line.group(10)
        line2items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
df2=pd.DataFrame(line2items)
df2

<re.Match object; span=(0, 49), match='3243387 032720 062720 75.00 P 75.00 3 Postage * 1>
<re.Match object; span=(0, 69), match='3243387 072620 072620 75.00 P 75.00 2 Overnight d>


,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,200,UnitedParcelService,200.01MLJ,3243387,032720,062720,75.00,P,75.00,,3,Postage,1
1,200,UnitedParcelService,200.01MLJ,3243387,072620,072620,75.00,P,75.00,,2,Overnight document delivery,1


In [437]:
line3items=[]
ndetails=re.compile(r'(^\d{6}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    line=ndetails.search(line)
    if line:
        invoice_number=line.group(1)
        inv_date=line.group(2)
        due_date=line.group(3)
        inv_amount=line.group(4)
        type_=line.group(5)
        net_amount=line.group(6)
        validation=line.group(7)
        Ref=line.group(8)
        Description=line.group(9)
        Code=line.group(10)
        line3items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
df3=pd.DataFrame(line3items)
df3
        


,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,202,"SoftwareTechnology,LLC",200.01MLJ,312205,082020,082520,490.00,P,490.00,,1,Software Maintenance Agreement Tabs3,1
1,202,"SoftwareTechnology,LLC",200.01MLJ,312206,082020,082520,239.00,P,239.00,,2,Software Maintenance Agreement Tabs3 General L...,1
2,202,"SoftwareTechnology,LLC",200.01MLJ,312207,082020,082520,239.00,P,239.00,,3,Software Maintenance Agreement Tabs3 Accounts ...,1
3,202,"SoftwareTechnology,LLC",200.01MLJ,312208,082020,082520,239.00,P,239.00,,4,Software Maintenance Agreement Tabs3 Trust Acc...,1
4,202,"SoftwareTechnology,LLC",200.01MLJ,312209,082020,082520,800.00,P,800.00,,5,Software Maintenance Agreement PracticeMaster ...,1
5,202,"SoftwareTechnology,LLC",200.01MLJ,312210,082020,082520,105.00,P,105.00,,6,Software Maintenance Agreement Tabs3 Taskbill,1


In [438]:
line4items=[]
clerkdetail=re.compile(r'(^\d{3}\.\d{2}[A-Z]*) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    line=clerkdetail.search(line)
    if line:
        voucher=line.group(1)
        invoice_number=''
        inv_date=line.group(2)
        due_date=line.group(3)
        inv_amount=line.group(4)
        type_=line.group(5)
        net_amount=line.group(6)
        validation=line.group(7)
        Ref=line.group(8)
        Description=line.group(9)
        Code=line.group(10)
        line4items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
line4items
df4=pd.DataFrame(line4items)
df4

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,203,ClerkoftheCountyCourt,200.01PAM,,042620,042620,225.00,P,225.00,,1,"Documentation Fee, Filing Fee, and Sheriff Ser...",1


In [439]:
line5items=[]
startingsix=re.compile(r'(^\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)

    line=startingsix.search(line)
    if line:
        inv_date=line.group(1)
        due_date=line.group(2)
        inv_amount=line.group(3)
        type_=line.group(4)
        net_amount=line.group(5)
        validation=line.group(6)
        Ref=line.group(7)
        Description=line.group(8)
        Code=line.group(9)
        line5items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
line5items
df5=pd.DataFrame(line5items)
df5
        

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,204,ClerkoftheDistrictCourt,200.01PAM,,032720,032720,75.00,P,75.00,,2,Filing fee,1
1,204,ClerkoftheDistrictCourt,200.01PAM,,052020,052020,75.00,P,75.00,,3,Filing fee,1
2,225,Jackson/Wylinda,200.01PAM,,122419,122419,750.00,P,750.00,Y,2,Holiday Party,1
3,225,Jackson/Wylinda,200.01PAM,,072620,072620,"1,500.00",P,"1,500.00",Y,1,Catering/Firm Retreat,1
4,300,ThomsonReutersPaymentCenter,200.01PAM,,042520,042520,4.00,P,4.00,,3,Online Legal Research,1
5,300,ThomsonReutersPaymentCenter,200.01PAM,,052420,052420,75.00,P,75.00,,1,Online Legal Research,1
6,300,ThomsonReutersPaymentCenter,200.01PAM,,062520,062520,40.00,P,40.00,,4,Online Legal Research,1
7,300,ThomsonReutersPaymentCenter,200.01PAM,,072920,072920,4.00,P,4.00,,2,Online Legal Research,1
8,300,ThomsonReutersPaymentCenter,200.01PAM,,073020,073020,50.00,P,50.00,,5,Online Legal Research,1
9,300,ThomsonReutersPaymentCenter,200.01PAM,,073020,073020,50.00,P,50.00,,6,Online Legal Research,1


In [440]:
line6items=[]
check=re.compile(r'(^\d{3}\.\d{2}) ([A-Z]*) (\d{6}) (\d{6}) ([\d]+\.\d{2}) ([\d]+\.\d{2}) (\d) ([a-zA-Z].*) (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    line=check.search(line)
    if line:
        voucher=(line.group(1)+line.group(2))
        invoice_number=''
        inv_date=line.group(3)
        due_date=line.group(4)
        inv_amount=line.group(5)
        # type_=line.group(6)
        net_amount=line.group(6)
        # validation=line.group(8)
        Ref=line.group(7)
        Description=line.group(8)
        Code=line.group(9)
        line6items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
line6items
df6=pd.DataFrame(line6items)
df6

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,204,ClerkoftheDistrictCourt,850.00RON,,082020,082020,150.00,P,150.00,Y,1,Petition for Dissolution of Marriage Filing Fee,1


In [445]:
line7items=[]
numberstart=re.compile(r'(^\d{4}) (\d{7}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    
    line=numberstart.search(line)
    if line:
        voucher=line.group(1)
        invoice_number=line.group(2)
        inv_date=line.group(3)
        due_date=line.group(4)
        inv_amount=line.group(5)
        type_=line.group(6)
        net_amount=line.group(7)
        validation=line.group(8)
        Ref=line.group(9)
        Description=line.group(10)
        Code=line.group(11)
        line7items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
df7=pd.DataFrame(line7items)
df7

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,240,NELCO,5065,2094756,072620,082620,129.85,P,129.85,,1,Check order for APS and TAS Checks,1


In [460]:
line8items=[]
auto=re.compile(r'(^AUTO) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) (.*?) (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)

    line=auto.search(line)
    
    if line:
        voucher=line.group(1)
        # invoice_number=''
        inv_date=line.group(2)
        due_date=line.group(3)
        inv_amount=line.group(4)
        type_=line.group(5)
        net_amount=line.group(6)
        validation=line.group(7)
        Ref=line.group(8)
        Description=line.group(9)
        Code=line.group(11)
        line8items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
line8items
df8=pd.DataFrame(line8items)
df8

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,250,D&BRealEstateManagementCompany,AUTO,2094756,072220,072220,"7,500.00",P,"7,500.00",Y,1,"Monthly Rent (Including Electric, Gas & Garbage)",1
1,250,D&BRealEstateManagementCompany,AUTO,2094756,072620,072620,"7,500.00",P,"7,500.00",Y,2,"Monthly Rent (Including Electric, Gas & Garbage)",1
2,700,CleanAllJanitorialServices,AUTO,2094756,072220,072520,750.00,P,750.00,Y,1,Monthly Cleaning Charges (Bi-Weekly Service),1
3,700,CleanAllJanitorialServices,AUTO,2094756,072620,082520,750.00,,750.00,Y,2,Monthly Cleaning Charges (Bi-Weekly,1


In [473]:
line9items=[]
autokey=re.compile(r'(^\d{4}) ([A-Z].*) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    line=autokey.search(line)
    if line:
        print(line)
        voucher=line.group(1)
        invoice_number=line.group(2)
        inv_date=line.group(3)
        due_date=line.group(4)
        inv_amount=line.group(5)
        type_=line.group(6)
        net_amount=line.group(7)
        validation=line.group(8)
        Ref=line.group(9)
        Description=line.group(10)
        Code=line.group(11)
        line9items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
df9=pd.DataFrame(line9items)
df9

<re.Match object; span=(0, 67), match='5074 KEY 081320 081320 10.00 10.00 N 3 Duplicate >


,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,250,D&BRealEstateManagementCompany,5074,KEY,081320,081320,10.00,,10.00,N,3,Duplicate Key for building,1


In [477]:
line10items=[]
last=re.compile(r'(\d{12}) (\d{6}) (\d{6}) (\d{2}\.\d{2}) (\d{2}\.\d{2}) (\d) ([A-Z].*) (\d)')
for line in text.split('\n'):
    if vend_re.match(line):
        vend_num,*vend_desc=line.split()
        vend_desc=''.join(vend_desc)
    line=last.search(line)
    if line:
        # voucher=line.group(1)
        invoice_number=line.group(1)
        inv_date=line.group(2)
        due_date=line.group(3)
        inv_amount=line.group(4)
        type_=line.group(5)
        net_amount=line.group(7)
        # validation=line.group(8)
        Ref=line.group(6)
        Description=line.group(7)
        Code=line.group(8)
        line10items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
line10items
df10=pd.DataFrame(line10items)
df10

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,400,SprintLocal&LongDistance,5074,402310760011,082020,082520,68.96,68.96,November Cell Phone Charges,N,1,November Cell Phone Charges,1


In [479]:
finaldf=pd.concat([df,df2,df3,df4,df5,df6,df7,df8,df9,df10],ignore_index=True)
finaldf

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,200,UnitedParcelService,200.01MLJ,3243387,032620,062620,75.00,P,75.00,,1,Overnight document delivery,1
1,200,UnitedParcelService,200.01MLJ,3243387,032720,062720,75.00,P,75.00,,3,Postage,1
2,200,UnitedParcelService,200.01MLJ,3243387,072620,072620,75.00,P,75.00,,2,Overnight document delivery,1
3,202,"SoftwareTechnology,LLC",200.01MLJ,312205,082020,082520,490.00,P,490.00,,1,Software Maintenance Agreement Tabs3,1
4,202,"SoftwareTechnology,LLC",200.01MLJ,312206,082020,082520,239.00,P,239.00,,2,Software Maintenance Agreement Tabs3 General L...,1
5,202,"SoftwareTechnology,LLC",200.01MLJ,312207,082020,082520,239.00,P,239.00,,3,Software Maintenance Agreement Tabs3 Accounts ...,1
6,202,"SoftwareTechnology,LLC",200.01MLJ,312208,082020,082520,239.00,P,239.00,,4,Software Maintenance Agreement Tabs3 Trust Acc...,1
7,202,"SoftwareTechnology,LLC",200.01MLJ,312209,082020,082520,800.00,P,800.00,,5,Software Maintenance Agreement PracticeMaster ...,1
8,202,"SoftwareTechnology,LLC",200.01MLJ,312210,082020,082520,105.00,P,105.00,,6,Software Maintenance Agreement Tabs3 Taskbill,1
9,203,ClerkoftheCountyCourt,200.01PAM,,042620,042620,225.00,P,225.00,,1,"Documentation Fee, Filing Fee, and Sheriff Ser...",1


In [480]:
finaldf.inv_date=pd.to_datetime(finaldf.inv_date)
finaldf.due_date=pd.to_datetime(finaldf.due_date)
finaldf

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,200,UnitedParcelService,200.01MLJ,3243387,2020-03-26,2020-06-26,75.00,P,75.00,,1,Overnight document delivery,1
1,200,UnitedParcelService,200.01MLJ,3243387,2020-03-27,2020-06-27,75.00,P,75.00,,3,Postage,1
2,200,UnitedParcelService,200.01MLJ,3243387,2020-07-26,2020-07-26,75.00,P,75.00,,2,Overnight document delivery,1
3,202,"SoftwareTechnology,LLC",200.01MLJ,312205,2020-08-20,2020-08-25,490.00,P,490.00,,1,Software Maintenance Agreement Tabs3,1
4,202,"SoftwareTechnology,LLC",200.01MLJ,312206,2020-08-20,2020-08-25,239.00,P,239.00,,2,Software Maintenance Agreement Tabs3 General L...,1
5,202,"SoftwareTechnology,LLC",200.01MLJ,312207,2020-08-20,2020-08-25,239.00,P,239.00,,3,Software Maintenance Agreement Tabs3 Accounts ...,1
6,202,"SoftwareTechnology,LLC",200.01MLJ,312208,2020-08-20,2020-08-25,239.00,P,239.00,,4,Software Maintenance Agreement Tabs3 Trust Acc...,1
7,202,"SoftwareTechnology,LLC",200.01MLJ,312209,2020-08-20,2020-08-25,800.00,P,800.00,,5,Software Maintenance Agreement PracticeMaster ...,1
8,202,"SoftwareTechnology,LLC",200.01MLJ,312210,2020-08-20,2020-08-25,105.00,P,105.00,,6,Software Maintenance Agreement Tabs3 Taskbill,1
9,203,ClerkoftheCountyCourt,200.01PAM,,2020-04-26,2020-04-26,225.00,P,225.00,,1,"Documentation Fee, Filing Fee, and Sheriff Ser...",1


In [510]:
with pdfplumber.open(pdf_path) as pdf:
    pages=pdf.pages
    for page in pages:
        text=page.extract_text()
        vend_re=re.compile(r'\d{3} [A-Z].*')
        somedetails=re.compile(r'(^\d{3}\.\d{2}[A-Z].*) (\d{7}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
        moredetails=re.compile(r'(^\d{7}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
        line1items=[]
        line2items=[]
        for line in text.split('\n'):
            if vend_re.match(line):
                vend_num,*vend_desc=line.split()
                vend_desc=''.join(vend_desc)
            
            if somedetails.search(line):
                voucher=somedetails.search(line).group(1)
                invoice_number=somedetails.search(line).group(2)
                inv_date=somedetails.search(line).group(3)
                due_date=somedetails.search(line).group(4)
                inv_amount=somedetails.search(line).group(5)
                type_=somedetails.search(line).group(6)
                net_amount=somedetails.search(line).group(7)
                validation=somedetails.search(line).group(8)
                Ref=somedetails.search(line).group(9)
                Description=somedetails.search(line).group(10)
                Code=somedetails.search(line).group(11)
                line1items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
            if moredetails.search(line):
                invoice_number=moredetails.search(line).group(1)
                inv_date=moredetails.search(line).group(2)
                due_date=moredetails.search(line).group(3)
                inv_amount=moredetails.search(line).group(4)
                type_=moredetails.search(line).group(5)
                net_amount=moredetails.search(line).group(6)
                validation=moredetails.search(line).group(7)
                Ref=moredetails.search(line).group(8)
                Description=moredetails.search(line).group(9)
                Code=moredetails.search(line).group(10)
                line2items.append(INV(vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code))
        if len(line1items)>0: 
            df=pd.DataFrame(line1items)
        if len(line2items)>0:
            df2=pd.DataFrame(line2items)
        fin=pd.concat([df,df2])
            
# df
#                 # print(vend_num,vend_desc)
# df=pd.DataFrame(line1items)
# df
        
    

        
    

In [511]:
fin

,vend_num,vend_desc,voucher,invoice_number,inv_date,due_date,inv_amount,type_,net_amount,validation,Ref,Description,Code
0,200,UnitedParcelService,200.01MLJ,3243387,032620,062620,75.00,P,75.00,,1,Overnight document delivery,1
0,200,UnitedParcelService,200.01MLJ,3243387,032720,062720,75.00,P,75.00,,3,Postage,1
1,200,UnitedParcelService,200.01MLJ,3243387,072620,072620,75.00,P,75.00,,2,Overnight document delivery,1


In [508]:
with pdfplumber.open(pdf_path) as pdf:
    pages=pdf.pages
    for page in pages:
        text=page.extract_text()
        pattern = re.compile(r'^(\d{3}\.\d{2}[A-Z].*)? (\d{7}) (\d{6}) (\d{6}) ([\d,]+\.\d{2}) ([\sP]*)([\d,]+\.\d{2}) ([YN ]*)(\d) ([a-zA-Z].*) [*] (\d)')
        matches = pattern.findall(text)
        line_items=[]
        line_items += [INV(*match) for match in matches]

    if len(line_items) > 0:
        # df = pd.DataFrame(line_items)
        print(line_items)

In [507]:
df

""
